In [ ]:
# use this notebook for rough drafts of stuff

# Imports

In [ ]:
from LightPipes import *

from functions.colours          import *
from functions.measurement      import *
from functions.optical_modes    import *
from functions.plotting         import *
from functions.propagation      import *

import aotools
# import cmath

pmap, imap, customColoursBGY, customColoursViridis = colours()

# Creating channel and propagation

## Create channel and prop OAM

In [ ]:
#Simulation Parameters
size = 45*cm
N = 192

lensSize=size/4 # Radius

#Beam Parameters
wavelength = 633*nm
w0=6.67/1.5*cm # Radius

#Propagataion Params
z=5000*m
numberOfPhaseScreens=3

#Turbulence Parameters
C2_n = { # These can be toyed around with
    'WeakestTurb' : 1e-19,
    'WeakerTurb' : 1e-18,
    'WeakTurb' : 1e-17,
    'MidWeakerTurb' : 1.5e-17,
    'MidWeakTurb' : 1e-16,
    'MidTurb' : 1e-15,
    'StrongTurb' : 1e-14,
    'StrongerTurb' : 1e-13 
}

TurbStrength = 'MidTurb'

r0 = pow(0.423*pow(2*np.pi/wavelength,2)*C2_n[TurbStrength]*z/numberOfPhaseScreens,-3/5)
# Turbulence=2
# r0=size/Turbulence

phase_screen_seed=47

abbs=[aotools.turbulence.phasescreen.ft_sh_phase_screen(r0,N,size/N,100,0.001) for i in range(numberOfPhaseScreens)]

#Default Beam
F=Begin(size,wavelength,N)

In [ ]:
beams=[propChannel(OAM(F,w0,i),z,numberOfPhaseScreens,0) for i in progress(ell(4))]
errorBeams=[propChannel(OAM(F,w0,i),z,abbs) for i in progress(ell(4))]


plt.show(plotCrosstalk(crosstalk(beams,errorBeams)))
print(f'Error rate of {beamsError(beams,errorBeams)}')

plt.show(plot_beam(errorBeams,rows=1,aperature=size/3))
plt.show(plot_beam(beams,rows=1,aperature=size/3))

## propagate a single pixel with no turbulence

In [ ]:
intens = np.zeros((N, N))
intens[N//2][N//2] = 1

#FieldOut = propChannel(SubIntensity(FieldIn, intens), z, abbs)
#FieldOut = propChannel(CircAperture(SubIntensity(FieldIn, intens),FieldIn.siz/2), z, abbs)
F2 = [CircAperture(propChannel(CircAperture(SubPhase(SubIntensity(F, intens),np.zeros((N,N))),R=lensSize), dist),R=lensSize) for dist in [1,1000,2000,3000,4000,5000]]

plt.show(plot_beam(F2,aperature=lensSize,dpi=3*N))


## propagate all pixels

In [ ]:
# if __name__ == "__main__":
#     FieldsOut, endFields = parallelpropagatePixels(F, N, z, lensSize, abbs)
#     print(f"Successfully processed {len(endFields)} pixels.")

# Infinite Phase Screens

## General definitions

In [ ]:
#Simulation Parameters
size = 45*cm
N = 192

## Kolmogerov statistics

In [ ]:
from aotools.turbulence.infinitephasescreen import PhaseScreenKolmogorov
# Note that i set L0=75, if you try L0=100 you get some error at least for N=64. For N=64 we need L0<90 from my testing
# N=128 we need L0<41. Actually using L0=40 makes it look totally fucked lol. Even L0=30 is messed. Using L0=25 is nice... looks like using a small L0 with more resolution is the fix
L0 = N/2 * size / N * 50
inf_ps = PhaseScreenKolmogorov(nx_size=N, pixel_scale=size/N, r0=r0, L0=L0,random_seed=phase_screen_seed, stencil_length_factor=4)
inf_ps1 = inf_ps.scrn
inf_ps2 = PhaseScreenKolmogorov.get_new_row(inf_ps)

for i in range(0,int(N*100)):
    inf_ps3 = PhaseScreenKolmogorov.add_row(inf_ps)

In [ ]:
# plt.imshow(wrap_to_pi(inf_ps3), cmap=pmap, vmin=-np.pi, vmax=np.pi)
# plt.colorbar()

In [ ]:
# plt.imshow(wrap_to_pi(inf_ps3), cmap=pmap, vmin=-np.pi, vmax=np.pi)
# plt.colorbar()

## Von Karman statistics

In [ ]:
# Infinite phase screens but with Von Karmon statistics

from aotools.turbulence.infinitephasescreen import PhaseScreenVonKarman
L0 = N/2 * size / N * 50 # outer scale
inf_ps = PhaseScreenVonKarman(nx_size=N, pixel_scale=size/N, r0=r0, L0=L0, random_seed=phase_screen_seed)
inf_ps1 = inf_ps.scrn.copy()

# for i in range(0,int(N*10)):
#     inf_ps2 = PhaseScreenVonKarman.add_row(inf_ps)

# print(inf_ps.scrn)
# print('\n\n\n')
inf_ps.scrn[:] = np.flipud(inf_ps.scrn)
# print(inf_ps.scrn)
for _ in range(0, 20):
    inf_ps4 = PhaseScreenVonKarman.add_row(inf_ps)
inf_ps4 = np.flipud(inf_ps4)

In [ ]:
# plt.imshow(wrap_to_pi(inf_ps1), cmap=pmap, vmin=-np.pi, vmax=np.pi)
# plt.colorbar()

In [ ]:
# plt.imshow(wrap_to_pi(inf_ps2), cmap=pmap, vmin=-np.pi, vmax=np.pi)
# plt.colorbar()